# Installing libraries

In [1]:
!pip install transformers accelerate peft bitsandbytes "qwen-vl-utils[decord]==0.0.8"
# or use the 2U1 repo's environment.yaml/requirements.txt for comprehensive setup

# Fine-Tuning

In [32]:
!torchrun --nproc_per_node=1 run_finetune.py \
  --model qwen/Qwen2.5-VL-7B-Instruct \
  --data_dir test_data/ \
  --output_dir output/qwen-vet \
  --lora_rank 8 \
  --num_train_epochs 3 \
  --batch_size 3 \
  --grad_accum 2

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Loading checkpoint shards: 100%|██████████████████| 5/5 [00:14<00:00,  2.94s/it]
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
  0%|                     

In [13]:
from typing import List
import json

def load_jsonl(path: str) -> List[dict]:
    records = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records

In [15]:
rec = load_jsonl("test_data/train.jsonl")[:2]

In [21]:
rec[0]

{'image': '/teamspace/studios/this_studio/.cache/kagglehub/datasets/youssefmohmmed/dogs-skin-diseases-image-dataset/versions/1/test/Dermatitis/May-is-allergy-awareness-month-here-at-The-Shot-Spot_02176_jpg.rf.9e9d261224278a3327050d94a55dd859.jpg',
 'conversations': [{'from': 'human',
   'value': "My dog has these red, inflamed spots on his skin, and every time we think we've treated them, a new one pops up. What could be wrong with him?"},
  {'from': 'gpt',
   'value': "It sounds like your dog may be suffering from dermatitis, which is a common skin condition. This can be caused by various factors like allergies, infections, parasites, or irritants. \n\nSymptoms can include redness, itching, inflammation, bumps, or sores on the skin. \n\nFor home remedies, you can try giving your dog a cool bath with oatmeal or a veterinary-approved hypoallergenic shampoo. Applying a cool compress to the affected area might also offer some relief. \n\nPrevention involves identifying and removing potent

In [19]:
from PIL import Image

image_path = rec[0]['image']  # Replace with the actual path
img = Image.open(image_path)

In [22]:
from PIL import Image
import json
import os

jsonl_path = "test_data/train.jsonl"  # adjust as needed
bad = []
ok = []

with open(jsonl_path) as f:
    for i, line in enumerate(f):
        try:
            rec = json.loads(line)
            image_path = rec["image"]
            assert os.path.exists(image_path), f"File not found: {image_path}"
            img = Image.open(image_path).convert("RGB")
            img.verify()  # Check for corrupted files
            ok.append(image_path)
        except Exception as e:
            bad.append((i, image_path, str(e)))

print(f"\n✅ Good images: {len(ok)}")
print(f"❌ Bad images: {len(bad)}")
if bad:
    print("Examples of bad ones:")
    for item in bad[:5]:
        print(item)



✅ Good images: 878
❌ Bad images: 0
